In [ ]:
# Sample code for building a LeNet from scratch 
# Trained and tested on MNIST

In [59]:
import numpy as np
import torch
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Subset, DataLoader, TensorDataset
import random
import os

In [60]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [61]:
# Define LeNet Architecture
class LeNet(nn.Module):
    """
    LeNet-5

    Architecture: 2 convolutional layers followed by 3 fully connected layers.
    Reference: LeCun et al., "Gradient-Based Learning Applied to Document
    Recognition", Proceedings of the IEEE, 1998.

    Note: The original LeNet-5 uses tanh activations; here we use ReLU
    following modern convention. 
    """
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0)
        self.conv2 = nn.Conv2d(6, 12, kernel_size=5, stride=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.relu = nn.ReLU()
        self.fc = nn.Linear(12 * 4 * 4, 10)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 12 * 4 * 4)
        x = self.fc(x)
        return x

In [62]:
# Training Loop
def train(model, data_tr, criterion, optimizer, epochs=10, log=True):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for images, labels in data_tr:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        if log:
          print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(train_loader):.4f}")

# Evaluate model
def evaluate(model, data_te):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in data_te:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

In [63]:
# Load the full MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

In [64]:
# Initialize model, loss function, and optimizer
model = LeNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training LeNet
train(model, train_loader, criterion, optimizer, epochs=10)

Epoch [1/10], Loss: 0.3266
Epoch [2/10], Loss: 0.0973
Epoch [3/10], Loss: 0.0712
Epoch [4/10], Loss: 0.0605
Epoch [5/10], Loss: 0.0519
Epoch [6/10], Loss: 0.0464
Epoch [7/10], Loss: 0.0416
Epoch [8/10], Loss: 0.0392
Epoch [9/10], Loss: 0.0357
Epoch [10/10], Loss: 0.0341


In [65]:
# Testing the trained LeNet model
accuracy = evaluate(model, test_loader)
print(f"Test Accuracy: {accuracy:.2f}%")

Test Accuracy: 98.87%
